todo:
- email SS to ask if we can narrow api query & if they mind if we can't & get huge response
- we don't want to have older pubs which wouldn't have bi-direcitonal relaitonships

In [1]:
import requests
import json
import math

In [2]:
ssAPIkey = 'qZWKkOKyzP5g9fgjyMmBt1MN2NTC6aT61UklAiyw' # approved for 100 req/sec
headers = {'x-api-key': ssAPIkey}
ssBASE = 'http://api.semanticscholar.org/graph/v1/paper/'
fields = "title,year,authors,referenceCount,isOpenAccess,fieldsOfStudy,s2FieldsOfStudy"

search_terms = '"universal design for learning" or "udl"'
limit = 100
offset = 0

### Fetch Search Results

In [3]:
def fetch_papers(search_terms, limit, offset, fields):
  ssENDPOINTsearch = 'search?' + \
                    f'&query={search_terms}' + \
                    f'&limit={limit}&offset={offset}' + \
                    f'&fields={fields}'
  resp = requests.get(ssBASE + ssENDPOINTsearch, headers=headers)
  data = resp.json()['data']
  return resp, data


def write_paper(data, nodeType='Target'):
  with open('udl.jsonl','a') as fout:
    for paper in data:
      paper['nodeType'] = nodeType
      json.dump(paper, fout)
      fout.write('\n') 

resp, data = fetch_papers(search_terms, limit, offset, fields)
write_paper(data, nodeType='Target')
print(f"search target result count: {resp.json()['total']}")

while 'next' in resp.json():
  try:
    resp, data = fetch_papers(search_terms, limit, resp.json()['next'], fields)
    write_paper(data, nodeType='Target')
  except Exception as e: 
    print(e)
    print('API maxes out around 10k')
    break

num_lines = sum(1 for line in open('udl.jsonl'))
print(f'target papers written: {num_lines}')

search target result count: 948
target papers written: 948


### Fetch References

In [4]:
def fetch_links(paperId, link_type):
  resp = requests.get(ssBASE + 
                      paperId + f'/{link_type}s' + 
                      '?fields=' + fields, 
                      headers=headers)
  data = resp.json()['data']
  return resp, data


def write_link(data, nodeType, target):
  with open('udl_links.jsonl','a') as fout:
    data['nodeType'] = nodeType
    data['targetNode'] = target
    json.dump(data, fout)
    fout.write('\n') 


with open('udl.jsonl','r') as targetFile:
  for line in targetFile:
    line = json.loads(line)
    resp, data = fetch_links(line['paperId'], 'reference')
    for ref in data:
      write_link(ref['citedPaper'], nodeType='Reference', target=line['paperId']) # out links (papers cited by target)
    resp, data = fetch_links(line['paperId'], 'citation')
    for cit in data:
      write_link(cit['citingPaper'], nodeType='Citation', target=line['paperId']) # in links (papers citing target)
